In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn
import sklearn

from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import sklearn.discriminant_analysis
import sklearn.neighbors

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn import cross_validation
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale

%matplotlib inline

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
boston = pd.read_csv('Boston.csv')
boston.head()
boston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [8]:
mu_hat = np.mean(boston['medv'])
mu_hat
# The population mean for 'medv' is approximately 22.532806324110677.

22.532806324110677

In [9]:
std_err_hat = np.std(boston.iloc[:, 13]) / np.sqrt(len(boston.index))
std_err_hat

0.40845693469728661

In [33]:
realizations = np.zeros((1000, 506, 14))
np.random.seed(seed = 50)

for i in range(1000) :
    indices = np.random.choice(np.arange(0, 506, 1), size = 506, replace = True)
    for j in range(506) :
        realizations[i, j, :] = boston.iloc[indices[j], :]

bootstrap_means = np.zeros((1000, 1))
for num in range(1000) :
    bootstrap_means[num, 0] = np.mean(realizations[num, :, 13])

0.39511693250398283

In [41]:
# Now we have 1000 different 'realizations of history'. We can calculate the standard error
# of the population mean for 'medv' as follows:
std_err_bs_hat = 0

for n in range(1000) :
    std_err_bs_hat = std_err_bs_hat + (bootstrap_means[n, 0] - np.mean(bootstrap_means[:, 0]))**2
    
std_err_bs_hat = np.sqrt((1 / 999) * std_err_bs_hat)
std_err_bs_hat

0.39531463926266647

In [42]:
mu_bs_hat = np.mean(bootstrap_means[:, 0])
lower_bound = mu_bs_hat - 2 * std_err_bs_hat
upper_bound = mu_bs_hat + 2 * std_err_bs_hat
print(lower_bound, upper_bound)
# Based on bootstrap, the 95% confidence interval for population mean is [21.7253628163, 23.3066213734].

21.7253628163 23.3066213734


In [17]:
med_hat = np.median(boston['medv'])
med_hat
# The population median for 'medv' is approximately 21.199999999999999.

21.199999999999999

In [40]:
bootstrap_medians = np.zeros((1000, 1))
for num1 in range(1000) :
    bootstrap_medians[num1, 0] = np.median(realizations[num1, :, 13])

In [37]:
# Now we have 1000 different 'realizations of history'. We can calculate the standard error
# of the population mean for 'medv' as follows:
std_err_bs_hat1 = 0

for n in range(1000) :
    std_err_bs_hat1 = std_err_bs_hat1 + (bootstrap_medians[n, 0] - np.mean(bootstrap_medians[:, 0]))**2
    
std_err_bs_hat1 = np.sqrt((1 / 999) * std_err_bs_hat1)
std_err_bs_hat1

0.37751326634472882

In [43]:
med_bs_hat1 = np.mean(bootstrap_medians[:, 0])
lower_bound1 = med_bs_hat1 - 2 * std_err_bs_hat1
upper_bound1 = med_bs_hat1 + 2 * std_err_bs_hat1
print(med_bs_hat1)
print(lower_bound1, upper_bound1)
# Based on bootstrap, the 95% confidence interval for the median is [20.4274734673, 21.9375265327].
# The value calculated for the population median of 'medv' falls inside of the 95% CI found with bootstrap.
# Not only does the median fall inside of the CI, it is almost in the center of the interval.
# Also, the standard error for the median of 'medv' is not too large.

21.16615
20.4111234673 21.9211765327
